<a href="https://colab.research.google.com/github/mdiogof/Contrata-Bot/blob/main/Contrata_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
%pip -q install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0

In [ ]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import Markdown, display
from IPython.display import display, HTML
import requests
import warnings
import json
import re

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate = lambda _: True))

In [ ]:
def agente_buscador(curso, local, cargo, data_de_hoje):
  buscador = Agent(
      name = "agente_buscador",
      model = "gemini-2.0-flash",
      instruction = """
      Você é um assistente de busca. Sua função é usar a ferramenta de pesquisa
      do Google (google_search) para encontrar vagas de emprego, baseando-se no
      curso informado abaixo. Considere também o local informado e o cargo
      desejado na empresa. Essa busca deve focar em ofertas de emprego atuais,
      de no máximo duas semanas antes da data de hoje. Confira se a vaga ainda
      está aberta e inclua a data de publicação de cada vaga.
      """,
      description = "Agente que encontra vagas de emprego no Google",
      tools = [google_search]
  )

  entrada_agente_buscador = f"Curso: {curso}\nLocal: {local}\nCargo: {cargo}\nData de hoje: {data_de_hoje}"
  vagas = call_agent(buscador, entrada_agente_buscador)
  return vagas

In [ ]:
def agente_seletor(vagas_encontradas, periodo):
    seletor = Agent(
        name="agente_seletor",
        model="gemini-2.0-flash",
        instruction="""
        Você é um agente seletor de vagas de emprego para estudantes do ensino superior.
        Sua função é refinar a seleção das vagas com base na relevância em relação ao período de curso informado.

        🔸 Responda apenas com um JSON puro como este:
        [
          {"titulo": "Estágio em Desenvolvimento - Empresa X", "link": "https://example.com/vaga1"},
          {"titulo": "Estágio em Hardware - Empresa Y", "link": "https://example.com/vaga2"}
        ]

        ⚠️ Não inclua texto explicativo, markdown ou código markdown (```json).
        ⚠️ Não invente vagas, use apenas as fornecidas.
        """,
        description="Agente que refina a busca de vagas de emprego",
        tools=[google_search]
    )

    entrada = f"Vagas: {vagas_encontradas}\nPeríodo: {periodo}"
    resposta = call_agent(seletor, entrada)

    # Extração de JSON usando expressão regular
    match = re.search(r'\[.*?\]', resposta, re.DOTALL)
    if match:
        try:
            vagas_json = json.loads(match.group())
            return vagas_json
        except json.JSONDecodeError:
            print("⚠️ Erro ao decodificar JSON.")
            return []
    else:
        print("⚠️ Nenhum JSON válido encontrado na resposta.")
        return []

In [ ]:
def exibidor_de_links(vagas_selecionadas):
    # Gerar os botões HTML com base nas vagas
    botoes_html = ""
    for vaga in vagas_selecionadas:
        titulo = vaga["titulo"]
        link = vaga["link"]
        botoes_html += f"""
        <a href="{link}" target="_blank" style="text-decoration:none;">
            <button style="
                background-color:#4CAF50;
                color:white;
                padding:10px 20px;
                margin:5px;
                border:none;
                border-radius:5px;
                cursor:pointer;
                font-size:16px;
            ">{titulo}</button>
        </a>
        """

    # Exibir os botões
    return botoes_html

In [11]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("""Olá, você acessou o Contrata Bot!\n
Antes de te mostrar a oportunidade certa, vou precisar de algumas informações.""")
curso = input("Primeiro, digite o curso que você faz: ")
periodo = input("E em qual período de curso você está? ")
local = input("Agora, digite a região que você quer encontrar vagas: ")
cargo = input("Por fim, digite o cargo que você quer: ")

if not curso or not local or not cargo:
  print("Você precisa preencher todos os campos!")
else:
  print("Perfeito, vou procurar vagas para você!")

  vagas = agente_buscador(curso, local, cargo, data_de_hoje)
  print("\n--- Vagas encontradas pelo agente 1 (Buscador) ---\n")
  display(to_markdown(vagas))
  print("----------------------------------------------------\n")

  vagas_selecionadas = agente_seletor(vagas, periodo)
  print("\n--- Essas são as vagas mais relevantes para você no momento---\n")
  display(HTML(exibidor_de_links(vagas_selecionadas)))

Olá, você acessou o Contrata Bot!

Antes de te mostrar a oportunidade certa, vou precisar de algumas informações.
Primeiro, digite o curso que você faz: engenharia de computação
E em qual período de curso você está? 1º semestre
Agora, digite a região que você quer encontrar vagas: Sorocaba-SP
Por fim, digite o cargo que você quer: estágio
Perfeito, vou procurar vagas para você!

--- Vagas encontradas pelo agente 1 (Buscador) ---



> Ok, vou procurar por vagas de estágio em Engenharia de Computação em Sorocaba-SP, com data de publicação de no máximo duas semanas.
> 
> 
> Encontrei algumas opções de vagas de estágio em Engenharia da Computação em Sorocaba, SP. É importante verificar a data de publicação para garantir que ainda estejam abertas.
> 
> **Vagas encontradas:**
> 
> *   **Prefeitura de Sorocaba e CIEE:** Estão com inscrições abertas para processo seletivo de vagas de estágio em diversas áreas, incluindo Engenharia da Computação. As inscrições vão até 17 de janeiro de 2022. Os estagiários contratados receberão bolsa-auxílio de R$ 720 a R$ 900, além de auxílio-transporte de R$ 184,80. A carga horária é de 4 a 5 horas diárias. Para se inscrever, acesse: [https://pp.ciee.org.br/vitrine/3686/detalhe](https://pp.ciee.org.br/vitrine/3686/detalhe).
> *   **Indeed:** No dia 12 de maio de 2025, foram listadas 3 vagas de Engenharia da Computação em Sorocaba, SP. As vagas são para Analista de Testes II, Analista de Desenvolvimento de Software I e Analista de Desenvolvimento de Software III - Full Stack na Flex LTD.
> *   **Jooble:** Foram encontradas 934 vagas de estágio em Engenharia da Computação em Sorocaba, SP. Algumas das vagas listadas são: Estágio em Engenharia Civil - Obras na EKV Construtora (há 1 dia), Estágio em Engenharia na Facens (há 1 dia), Estágio em Engenharia de Produção na Employer RH (há 2 meses).
> 
> **Outras oportunidades:**
> 
> *   **Embraer:** A Embraer está com inscrições abertas para o programa de estágio em São José dos Campos (SP), com 200 vagas para estudantes de Tecnologia, Administração e Engenharia. A bolsa-auxílio pode chegar até R$ 2.400, além de benefícios.
> 
> **Observações:**
> 
> *   A pesquisa retornou vagas com datas diversas. É fundamental verificar a data de publicação e confirmar se a vaga ainda está disponível antes de se candidatar.
> *   O site da prefeitura de Sorocaba também pode ter informações sobre programas de estágio.


----------------------------------------------------


--- Essas são as vagas mais relevantes para você no momento---

